# Generate top of the book & trades using dedicated service - F#

### Overview
Retrieving trades and top of the book data points over a look back period is a valuable indicator to highlight market activity. It can be correlated with a set of technical indicators to better analyze market movements and build a decision making parameter.

This sample is designed to request simultaneously trades and top of the book and illustrate market activity over a look back period.

This sample demonstrates how to request and plot from a dedicated data service **on-demand** trades and best bid/ask over the time.

### Inputs/outputs
Trades and best Bid/Ask extraction sample requires instrument's identifier, date time intervals as per inputs and returns top fo the book the occured trades data points.

### Services used
This sample uses *gRPC requests* in order to retrieve trades and top of the book from the hosted service. The queried endpoint in this script are:
* *TickTradesAndBookService*: to directly retrieve trades and top of the book tick data from the server.

### Packages required
1. Systemathics packages:
    * *Systemathics.Apis.Type.Shared.V1*
    * *Systemathics.Apis.Services.Tick.V1*
2. Open source packages
    * *Google.Protobuf.WellKnownTypes*
    * *Google.Type*
    * *Grpc.Net.Client*
    * *Grpc.Core*
    * *XPlot.Plotly* as per display package
    
***

# Run Top of the book & Trades sample

### Step 1: Install packages

In [ ]:
#i "nuget: file:///home/jovyan/.nuget/packages/"
#i "nuget: https://api.nuget.org/v3/index.json"
#r "nuget: Systemathics.Apis, 0.9.*-pre*"
#r "nuget: XPlot.Plotly.Interactive"
#r "nuget: TaskBuilder.fs"

In [ ]:
open Systemathics.Apis.Helpers
open Systemathics.Apis.Type.Shared.V1
open Systemathics.Apis.Services.Tick.V1
open Google.Protobuf.WellKnownTypes
open Google.Type
open Grpc.Net.Client
open Grpc.Core
open XPlot.Plotly
open XPlot.Plotly.Interactive

### Step 2: Prepare API requests
The following code snippets retrieve authentication token and prepare the API request by: opening the *channel* and adding the *token* to the request header:

In [ ]:
// Get token as metadata
let headers = TokenHelpers.GetTokenAsMetaData();

// Create communication channel
let channel = ChannelHelpers.GetChannel();

### Step 3: Create and process request
To request *tick trades and book* service, we need to specify:
* Instrument identifier
* Time period selection: select start and end dates
* Tick data request parameters

#### 3.1 Instrument selection

In [ ]:
let ticker = "AAPL"
let exchange = "BATS"

#### 3.2 Time period delimitation

In [ ]:
// Create time intervals
let start = new DateTime(2021, 03, 05)
let end' = new DateTime(2021, 03, 05)
let dateIntervals = 
  new DateInterval(
    StartDate = new Date ( Year = start.Year, Month = start.Month, Day = start.Day ),
    EndDate = new Date ( Year = end'.Year, Month = end'.Month, Day = end'.Day )
  )

// Build the market data request time interval (we are using Google date time format : we have to cast the dates)
// UTC time zone
let timeInterval = 
  new TimeInterval(
     StartTime = new TimeOfDay ( Hours = 14, Minutes = 00, Seconds = 00 ),
     EndTime = new TimeOfDay ( Hours = 14, Minutes = 30, Seconds = 00 )
  )

#### 3.3 Request creation
The following code snippet creates *gRPC client*, process request and returns the request reply:

In [ ]:
// Generate constraints based on the previous time selection:
let constraints = new Constraints() 
constraints.DateIntervals.Add(dateIntervals)
constraints.TimeIntervals.Add(timeInterval)

The following code snippets create requests for *trades* and *top of the book* data and instantiates the service:

In [ ]:
// Generate the tick trades and book request
let request = new TickTradesAndBookRequest( Constraints = constraints )
let identifier = new Identifier ( Exchange = exchange, Ticker = ticker )
request.Identifiers.Add( [| identifier |]) 

In [ ]:
// Instantiate the tick trades and book service
let service = new TickTradesAndBookService.TickTradesAndBookServiceClient(channel)

### Step 4: Visualize data

#### 4.1 Retrieve top of the book & trades

In [ ]:
// Get the trades and the book limits
open FSharp.Control.Tasks.V1
open System.Collections.Generic

let trades = List<Trade>()
let book = List<BookLimit>()

let isNotNull obj =
    obj |> isNull |> not

let iterAsync (t: IAsyncEnumerable<'T>) (f : 'T -> unit) =
    let task = 
        task {
            let e = t.GetAsyncEnumerator()
            let mutable go = true
            try
                let! step = e.MoveNextAsync()
                go <- step
                while go do 
                    f e.Current
                    let! step = e.MoveNextAsync()
                    go <- step
            finally
                e.DisposeAsync().AsTask().Wait()
        }
    task.Wait()

let split (ttb: TickTradesAndBookResponse) =
    if(ttb |> isNotNull && ttb.Data |> isNotNull) then
        if (ttb.Data.Book |> isNotNull) then
            book.Add ttb.Data.Book
        else if (ttb.Data.Trade |> isNotNull) then
            trades.Add ttb.Data.Trade

let call = service.TickTradesAndBook(request, headers)
let enumerator = call.ResponseStream.ReadAllAsync()

iterAsync enumerator split

// Display data
trades
|> display
book
|> display

#### 4.2 Plot top of the book & trade prices
Display *best bid*, *best ask* and *trade prices* over the look back period with the package of your choice.<br>*`XPlot.Plotly`* is used as per open source display package.
Plot a simple line graph with the prices as follows: 

In [ ]:
// Plot simultaneously Bid, Ask and trade prices
let layout = 
    new Layout.Layout(
        title = $"Best Bid/Ask prices and size for {ticker}",
        xaxis = new Xaxis(title = "Time"),
        yaxis = new Yaxis(
            title = "Price", 
            range = [ 118; 122],
            side = "left"
        ),
        yaxis2 = new Yaxis (
            title = "Volume",
            range =  [0 ; 12000],
            side = "right"
        )
    )

let data = 
    [
        new Scattergl(
            name = "AskPrice",
            mode = "lines",
            x = (book |> Seq.map (fun elt -> elt.TimeStamp.ToDateTime())),
            y = (book |> Seq.map (fun elt -> elt.Ask.Price))
        )
        new Scattergl(
            name = "BidPrice",
            mode = "lines",
            x = (book |> Seq.map (fun elt -> elt.TimeStamp.ToDateTime())),
            y = (book |> Seq.map (fun elt ->  elt.Bid.Price))
        )
        new Scattergl(
            name = "TradePrice",
            mode = "lines",
            x = (trades |> Seq.map (fun elt -> elt.TimeStamp.ToDateTime())),
            y = (trades |> Seq.map (fun elt -> elt.Price))
        )
        new Scattergl(
            name = "AskSize",
            mode = "bars",
            x = (book |> Seq.map (fun elt -> elt.TimeStamp.ToDateTime())),
            y = (book |> Seq.map (fun elt -> elt.Ask.Size)),
            yaxis = "y2"
        )
        new Scattergl(
            name = "BidSize",
            mode = "bars",
            x = (book |> Seq.map (fun elt -> elt.TimeStamp.ToDateTime())),
            y = (book |> Seq.map (fun elt -> elt.Bid.Size)),
            yaxis = "y2"
        )
        new Scattergl(
            name = "TradeSize",
            mode = "markers",
            x = (trades |> Seq.map (fun elt -> elt.TimeStamp.ToDateTime())),
            y = (trades |> Seq.map (fun elt ->  elt.Size)),
            yaxis = "y2"
        )
    ]

Chart.Plot(data, layout)
|> Chart.WithWidth 1500
|> Chart.WithHeight 800
|> display
